In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


In [2]:
df = sns.load_dataset('mpg')

In [3]:
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger


In [4]:
df.drop('name',axis = 1 , inplace = True)

In [5]:
df['origin'] = df.origin.map({'usa': 1 , 'japan' : 2 , 'europe': 3})

In [6]:
df.isna().sum()
df['horsepower'] = df.horsepower.fillna(df['horsepower'].median())


In [7]:
from sklearn.model_selection import train_test_split
X = df.drop('mpg',axis = 1)
y = df.mpg

In [8]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=1)

CV with hyperparamet

Grid Search CV

In [9]:
from sklearn.model_selection import GridSearchCV , RandomizedSearchCV
from sklearn.linear_model import Lasso , Ridge
from sklearn.metrics import r2_score

In [10]:
#define lasso model
lasso = Lasso()

In [11]:
lasso

Lasso()

In [12]:
param_grid = {'alpha': [0.1 , 1 , 10 , 100 , 0.01 , 0.001]}

In [13]:
grid_search = GridSearchCV(estimator=lasso , param_grid = param_grid , cv = 5 , scoring = 'r2' , verbose = 2 )

In [14]:
grid_search

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0.1, 1, 10, 100, 0.01, 0.001]}, scoring='r2',
             verbose=2)

In [15]:
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ...........................................alpha=10; total time=   0.0s
[CV] END ........................................

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0.1, 1, 10, 100, 0.01, 0.001]}, scoring='r2',
             verbose=2)

In [16]:
grid_search.best_params_

{'alpha': 0.1}

In [17]:
grid_search.best_score_

np.float64(0.7977657505718685)

In [18]:
grid_search.best_estimator_

Lasso(alpha=0.1)

In [19]:
y_pred_grid = grid_search.best_estimator_.predict(X_test)

In [20]:
r2_grid = r2_score(y_test , y_pred_grid)

In [21]:
r2_grid

0.8550584006185754

RandomizedSearchCV

In [26]:
Rand_search = RandomizedSearchCV(estimator=lasso , param_distributions = param_grid , cv = 5 , scoring = 'r2' , verbose = 2 ,n_iter=5)

In [27]:
Rand_search
Rand_search.fit(X_train,y_train)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END ........................................

RandomizedSearchCV(cv=5, estimator=Lasso(), n_iter=5,
                   param_distributions={'alpha': [0.1, 1, 10, 100, 0.01,
                                                  0.001]},
                   scoring='r2', verbose=2)

In [28]:
print(f'Best param {Rand_search.best_params_}')
print(f'Best estimator {Rand_search.best_estimator_}')
print(f'Best index {Rand_search.best_index_}')
print(f'Best Score {Rand_search.best_score_}')

Best param {'alpha': 0.01}
Best estimator Lasso(alpha=0.01)
Best index 0
Best Score 0.796619277359755


In [29]:
y_pred_rand = Rand_search.best_estimator_.predict(X_test)
r2_Rand = r2_score(y_test,y_pred_rand)
print(r2_Rand)

0.8561329535858035


# Task
Perform hyperparameter tuning for Ridge and ElasticNet regression models using both GridSearchCV and RandomizedSearchCV, similar to what was done for the Lasso model. Then, compare the best scores and R2 scores of all three models (Lasso, Ridge, and ElasticNet) from both tuning methods in a summary table.

## Gridsearchcv for ridge

### Subtask:
Perform GridSearchCV for the Ridge regression model.


**Reasoning**:
Instantiate a Ridge regression model, define the parameter grid, create a GridSearchCV object, fit the GridSearchCV object, and print the best parameters, best score, best estimator, make predictions on the test set, and calculate and print the R-squared score.



In [30]:
# Define Ridge model
ridge = Ridge()

# Define parameter grid (using the same as Lasso)
param_grid_ridge = {'alpha': [0.1, 1, 10, 100, 0.01, 0.001]}

# Create GridSearchCV object for Ridge
grid_search_ridge = GridSearchCV(estimator=ridge, param_grid=param_grid_ridge, cv=5, scoring='r2', verbose=2)

# Fit GridSearchCV to the training data
grid_search_ridge.fit(X_train, y_train)

# Print best parameters, best score, and best estimator
print(f'Best parameters for Ridge (GridSearchCV): {grid_search_ridge.best_params_}')
print(f'Best score for Ridge (GridSearchCV): {grid_search_ridge.best_score_}')
print(f'Best estimator for Ridge (GridSearchCV): {grid_search_ridge.best_estimator_}')

# Make predictions on the test set
y_pred_grid_ridge = grid_search_ridge.best_estimator_.predict(X_test)

# Calculate R-squared score on the test set
r2_grid_ridge = r2_score(y_test, y_pred_grid_ridge)
print(f'R2 score for Ridge (GridSearchCV) on test set: {r2_grid_ridge}')

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ...........................................alpha=10; total time=   0.0s
[CV] END ........................................

## Randomizedsearchcv for ridge

### Subtask:
Perform RandomizedSearchCV for the Ridge regression model.


**Reasoning**:
I will now perform `RandomizedSearchCV` for the Ridge regression model. I will create the `RandomizedSearchCV` object with the specified parameters, fit it to the training data, print the best results, and finally evaluate the best model on the test set by calculating the R-squared score.



In [31]:
# Create RandomizedSearchCV object for Ridge
rand_search_ridge = RandomizedSearchCV(estimator=ridge, param_distributions=param_grid_ridge, cv=5, scoring='r2', verbose=2, n_iter=5)

# Fit RandomizedSearchCV to the training data
rand_search_ridge.fit(X_train, y_train)

# Print best parameters, best estimator, and best score
print(f'Best param for Ridge (RandomizedSearchCV): {rand_search_ridge.best_params_}')
print(f'Best estimator for Ridge (RandomizedSearchCV): {rand_search_ridge.best_estimator_}')
print(f'Best Score for Ridge (RandomizedSearchCV): {rand_search_ridge.best_score_}')

# Use the best estimator to predict on the test set
y_pred_rand_ridge = rand_search_ridge.best_estimator_.predict(X_test)

# Calculate the R-squared score
r2_rand_ridge = r2_score(y_test, y_pred_rand_ridge)
print(f'R2 score for Ridge (RandomizedSearchCV) on test set: {r2_rand_ridge}')

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END ...........................................alpha=10; total time=   0.0s
[CV] END ...........................................alpha=10; total time=   0.0s
[CV] END ...........................................alpha=10; total time=   0.0s
[CV] END ...........................................alpha=10; total time=   0.0s
[CV] END ...........................................alpha=10; total time=   0.0s
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END ........................................alpha=0.001; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ........................................

## Gridsearchcv for elasticnet

### Subtask:
Perform GridSearchCV for the ElasticNet regression model.


**Reasoning**:
I will now perform `GridSearchCV` for the `ElasticNet` model. I will first import the `ElasticNet` model, define the parameter grid for `alpha` and `l1_ratio`, instantiate the `GridSearchCV` object, fit it to the training data, and then print the best parameters, score, and estimator, and finally calculate the R-squared score on the test set.



In [32]:
from sklearn.linear_model import ElasticNet

# Instantiate ElasticNet model
elastic_net = ElasticNet()

# Define parameter grid for ElasticNet
param_grid_en = {'alpha': [0.1, 1, 10, 100, 0.01, 0.001],
                 'l1_ratio': [0.2, 0.5, 0.7, 0.9]}

# Create GridSearchCV object for ElasticNet
grid_search_en = GridSearchCV(estimator=elastic_net, param_grid=param_grid_en, cv=5, scoring='r2', verbose=2)

# Fit GridSearchCV to the training data
grid_search_en.fit(X_train, y_train)

# Print best parameters, best score, and best estimator
print(f'Best parameters for ElasticNet (GridSearchCV): {grid_search_en.best_params_}')
print(f'Best score for ElasticNet (GridSearchCV): {grid_search_en.best_score_}')
print(f'Best estimator for ElasticNet (GridSearchCV): {grid_search_en.best_estimator_}')

# Make predictions on the test set
y_pred_grid_en = grid_search_en.best_estimator_.predict(X_test)

# Calculate R-squared score on the test set
r2_grid_en = r2_score(y_test, y_pred_grid_en)
print(f'R2 score for ElasticNet (GridSearchCV) on test set: {r2_grid_en}')

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END ............................alpha=0.1, l1_ratio=0.2; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.2; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.2; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.2; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.2; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.7; total time=   0.0s
[CV] END ............................alpha=0.1,

## Randomizedsearchcv for elasticnet

### Subtask:
Perform RandomizedSearchCV for the ElasticNet regression model.


**Reasoning**:
I will now perform `RandomizedSearchCV` for the `ElasticNet` model. I will first instantiate the `RandomizedSearchCV` object with the specified parameters, then fit it to the training data. After the fitting is complete, I will print the best parameters, estimator, and score. Finally, I will make predictions on the test set and calculate the R-squared score, storing it in the `r2_rand_en` variable.



In [33]:
# Create RandomizedSearchCV object for ElasticNet
rand_search_en = RandomizedSearchCV(estimator=elastic_net, param_distributions=param_grid_en, cv=5, scoring='r2', verbose=2, n_iter=10)

# Fit RandomizedSearchCV to the training data
rand_search_en.fit(X_train, y_train)

# Print best parameters, best estimator, and best score
print(f'Best param for ElasticNet (RandomizedSearchCV): {rand_search_en.best_params_}')
print(f'Best estimator for ElasticNet (RandomizedSearchCV): {rand_search_en.best_estimator_}')
print(f'Best Score for ElasticNet (RandomizedSearchCV): {rand_search_en.best_score_}')

# Use the best estimator to predict on the test set
y_pred_rand_en = rand_search_en.best_estimator_.predict(X_test)

# Calculate the R-squared score
r2_rand_en = r2_score(y_test, y_pred_rand_en)
print(f'R2 score for ElasticNet (RandomizedSearchCV) on test set: {r2_rand_en}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ............................alpha=0.1, l1_ratio=0.2; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.2; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.2; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.2; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.2; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ..............................alpha=1, l1_ratio=0.5; total time=   0.0s
[CV] END ..............................alpha=1, 

## Compare results

### Subtask:
Create a summary table to compare the best scores and R2 scores for Lasso, Ridge, and ElasticNet from both GridSearchCV and RandomizedSearchCV.


**Reasoning**:
I will now create a pandas DataFrame to summarize the results of the hyperparameter tuning for all three models (Lasso, Ridge, and ElasticNet) using both GridSearchCV and RandomizedSearchCV. The DataFrame will include the model name, tuning method, best cross-validation score, and the R2 score on the test set, as requested in the instructions. This will provide a clear and concise comparison of the performance of each model and tuning method.



In [34]:
# Create a dictionary with the results
results = {
    'Model': ['Lasso', 'Lasso', 'Ridge', 'Ridge', 'ElasticNet', 'ElasticNet'],
    'Tuning Method': ['GridSearchCV', 'RandomizedSearchCV', 'GridSearchCV', 'RandomizedSearchCV', 'GridSearchCV', 'RandomizedSearchCV'],
    'Best Score (CV)': [
        grid_search.best_score_,
        Rand_search.best_score_,
        grid_search_ridge.best_score_,
        rand_search_ridge.best_score_,
        grid_search_en.best_score_,
        rand_search_en.best_score_
    ],
    'R2 Score (Test)': [
        r2_grid,
        r2_Rand,
        r2_grid_ridge,
        r2_rand_ridge,
        r2_grid_en,
        r2_rand_en
    ]
}

# Create a DataFrame from the dictionary
comparison_df = pd.DataFrame(results)

# Display the comparison table
display(comparison_df)

,Model,Tuning Method,Best Score (CV),R2 Score (Test)
0,Lasso,GridSearchCV,0.797766,0.855058
1,Lasso,RandomizedSearchCV,0.796619,0.856133
2,Ridge,GridSearchCV,0.796485,0.856349
3,Ridge,RandomizedSearchCV,0.796485,0.856349
4,ElasticNet,GridSearchCV,0.797675,0.855043
5,ElasticNet,RandomizedSearchCV,0.797675,0.855043


## Summary:

### Data Analysis Key Findings
*   All three models (Lasso, Ridge, and ElasticNet) achieved very similar performance, with R2 scores on the test set all hovering around 0.855 to 0.856.
*   The best performing model on the test set was the Ridge model, with an R2 score of approximately 0.8563, regardless of the tuning method used.
*   The ElasticNet model found the best hyperparameters to be `alpha=0.1` and `l1_ratio=0.9`, resulting in a test R2 score of approximately 0.855.
*   The Ridge model identified the best hyperparameter `alpha` as 10, leading to a test R2 score of around 0.856.
*   For all models, both `GridSearchCV` and `RandomizedSearchCV` yielded almost identical results, indicating that for this particular dataset and hyperparameter space, the more exhaustive grid search did not provide a significant advantage over the more efficient randomized search.

### Insights or Next Steps
*   Since all three regularized regression models performed similarly, other factors such as model interpretability or computational efficiency could be considered for model selection.
*   Given the high performance of the linear models, it might be valuable to explore more complex non-linear models like Gradient Boosting or Random Forest to see if they can capture more complex relationships in the data and further improve the R2 score.
